In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.utils import resample
from collections import Counter

from custom_functions import read_df, split_train_test_NBest, remove_constant_columns, select_n_features
#, my_round, print_result, getQAlgo, aggregate_pred, weighted_average, 

In [2]:
'''
Methods: 
Index(['Address', 'Brain Method', 'Extensive Coupling', 'Feature Envy',
       'Intensive Coupling', 'Shortcircuiting Method', 'Shotgun Surgery',
       'ALD', 'ATFD', 'CALIN', 'CC', 'CCL', 'CDISP', 'CEXT', 'CFG Cyclo',
       'CINT', 'CM', 'CYCLO', 'DUPSTRING', 'EDUPLINES', 'FANIN', 'FANOUT',
       'FANOUTCLASS', 'FDP', 'HDUPLINES', 'IDUPLINES', 'LAA', 'LOC',
       'MAXNESTING', 'NOAV', 'NOEU', 'NOLV', 'NOP', 'NOVRM',
       'NoOfExternalClients', 'NoOfInternalClients', 'PCR',
       'average call generality', 'Unnamed: 38', 'Name', 'Unnamed: 39'],
      dtype='object')
      
      
Class: 
Index(['Address', 'Brain Class', 'Data Class', 'Futile Abstract Pipeline',
       'Futile Hierarchy', 'God Class', 'Hierarchy Duplication', 'Model Class',
       'Schizofrenic Class', 'ABUSEINH', 'AMW', 'ATFD', 'BOvM', 'BUR', 'CBO',
       'CC', 'CM', 'CRIX', 'DAC', 'DIT', 'EDUPCLS', 'FANOUT', 'FDP', 'GREEDY',
       'HDUPCLS', 'HIT', 'IDUPLINES', 'LOCC', 'NAS', 'NAbsM', 'NDU', 'NOA',
       'NOAM', 'NOD', 'NODD', 'NOM', 'NOPA', 'NProtM', 'NSPECM', 'NTempF',
       'NrBM', 'NrEC', 'NrFE', 'NrIC', 'NrSS', 'PNAS', 'SCHIZO', 'TCC', 'WMC',
       'WOC'],
      dtype='object')
'''

data_dir = "../datasets/codesmell/"
# model_dir = "savedModels/lucene/"

# data_files = os.listdir(data_dir)

# print(data_files[0])
class_labels = ['Brain Class', 'Data Class', 'Futile Abstract Pipeline', 'Futile Hierarchy', 'God Class', 'Model Class', 'Schizofrenic Class']
method_labels = ['Brain Method', 'Extensive Coupling', 'Feature Envy', 'Intensive Coupling', 'Shortcircuiting Method', 'Shotgun Surgery']

In [3]:
def my_resample(df):
    #it do not creates any artificial data, discards the majority class to match the number of minority
    df_true = df[df.iloc[:, -1] == True]    
    df_false = df[df.iloc[:, -1] == False]
    
    if len(df_false) > len(df_true):
        df_false = df_false.sample(n=len(df_true), replace=False)
        
    elif len(df_true) > len(df_false):
        df_true = df_true.sample(n=len(df_false), replace=False)

    # Concatenate the upsampled True class with the downsampled False class
    df_resampled = pd.concat([df_true, df_false])
    
    # Shuffle the DataFrame to randomize the order
    df_resampled = df_resampled.sample(frac=1).reset_index(drop=True)
    
    return df_resampled

In [4]:
filename = 'class'
df = read_df(data_dir + filename+'.csv', split=False)
print(df.shape)

for L in class_labels:
    print("Done:", L)
    Y = df[L].astype(int)
    X = df.iloc[:, 9:]
    
    df_new = pd.concat([X, Y], axis=1)
    print(df_new.shape, Counter(df_new.iloc[:, -1]))
    
    balanced_df = my_resample(df_new)
    print(balanced_df.shape, Counter(balanced_df.iloc[:, -1]))
    
    f10_df = select_n_features(balanced_df, 10)
    print(f10_df.shape, Counter(f10_df.iloc[:, -1]))
    
    ds_name = L.replace(' ', '_').lower()
    f10_df.to_csv(f"{data_dir}{filename}_balanced/{ds_name}.csv", index = False)
    print()

(373400, 50)
Done: Brain Class
(373400, 42) Counter({0: 372557, 1: 843})
(1686, 42) Counter({0: 843, 1: 843})
Index(['AMW', 'CBO', 'CRIX', 'FANOUT', 'LOCC', 'NOA', 'NOM', 'NrBM', 'TCC',
       'WMC'],
      dtype='object')
(1686, 11) Counter({0: 843, 1: 843})

Done: Data Class
(373400, 42) Counter({0: 351230, 1: 22170})
(44340, 42) Counter({0: 22170, 1: 22170})
Index(['AMW', 'CRIX', 'LOCC', 'NOA', 'NOAM', 'NOM', 'NOPA', 'TCC', 'WMC',
       'WOC'],
      dtype='object')
(44340, 11) Counter({0: 22170, 1: 22170})

Done: Futile Abstract Pipeline
(373400, 42) Counter({0: 373062, 1: 338})
(676, 42) Counter({0: 338, 1: 338})
Index(['BUR', 'DIT', 'HIT', 'LOCC', 'NAS', 'NOD', 'NODD', 'NOM', 'NProtM',
       'PNAS'],
      dtype='object')
(676, 11) Counter({0: 338, 1: 338})

Done: Futile Hierarchy
(373400, 42) Counter({0: 373036, 1: 364})
(728, 42) Counter({1: 364, 0: 364})
Index(['AMW', 'HIT', 'LOCC', 'NAbsM', 'NOD', 'NODD', 'NOM', 'NProtM', 'TCC',
       'WMC'],
      dtype='object')
(728, 11

In [23]:
data_type = 'method'
all_files = os.listdir(f"{data_dir}{data_type}_balanced/")

In [22]:
for data_file in all_files:
    file_to_read = f"{data_dir}{data_type}_balanced/{data_file}"
    df = read_df(file_to_read, split=False)
    print(df.shape, Counter(df.iloc[:, -1]))
    train_size = int(0.70 * len(df))
    train_df = df.iloc[0:train_size, :]
    test_df = df.iloc[train_size:, :]
    
    print(file_to_read)
    print(train_df.shape, Counter(train_df.iloc[:, -1]))
    print(test_df.shape, Counter(test_df.iloc[:, -1]))
    
    train_df.to_csv(f"{file_to_read[0:-4]}_train.csv", index = False)
    test_df.to_csv(f"{file_to_read[0:-4]}_test.csv", index = False)
    print()

(10646, 11) Counter({1: 5323, 0: 5323})
../datasets/codesmell/method_balanced/shortcircuiting_method.csv
(7452, 11) Counter({0: 3736, 1: 3716})
(3194, 11) Counter({1: 1607, 0: 1587})

(30560, 11) Counter({0: 15280, 1: 15280})
../datasets/codesmell/method_balanced/brain_method.csv
(21392, 11) Counter({0: 10747, 1: 10645})
(9168, 11) Counter({1: 4635, 0: 4533})

(39288, 11) Counter({0: 19644, 1: 19644})
../datasets/codesmell/method_balanced/shotgun_surgery.csv
(27501, 11) Counter({1: 13759, 0: 13742})
(11787, 11) Counter({0: 5902, 1: 5885})

(6240, 11) Counter({1: 3120, 0: 3120})
../datasets/codesmell/method_balanced/extensive_coupling.csv
(4368, 11) Counter({1: 2191, 0: 2177})
(1872, 11) Counter({0: 943, 1: 929})

(44846, 11) Counter({1: 22423, 0: 22423})
../datasets/codesmell/method_balanced/feature_envy.csv
(31392, 11) Counter({0: 15713, 1: 15679})
(13454, 11) Counter({1: 6744, 0: 6710})

(16172, 11) Counter({0: 8086, 1: 8086})
../datasets/codesmell/method_balanced/intensive_coupling.

In [29]:
class_file_names = ['futile_abstract_pipeline', 'god_class', 'futile_hierarchy', 'model_class', 
                    'brain_class', 'data_class', 'schizofrenic_class']
method_file_names = ['shortcircuiting_method', 'brain_method', 'shotgun_surgery', 'extensive_coupling', 
                     'feature_envy', 'intensive_coupling']

data_type = 'class'
for file in class_file_names:
    df = read_df(f"{data_dir}{data_type}_balanced/{file}_train.csv", split=False)
    print("train:", df.shape, Counter(df.iloc[:, -1]))
    
    df = read_df(f"{data_dir}{data_type}_balanced/{file}_test.csv", split=False)
    print("test:", df.shape, Counter(df.iloc[:, -1]))
    print()

train: (473, 11) Counter({0: 242, 1: 231})
test: (203, 11) Counter({1: 107, 0: 96})

train: (4972, 11) Counter({0: 2494, 1: 2478})
test: (2132, 11) Counter({1: 1074, 0: 1058})

train: (509, 11) Counter({0: 258, 1: 251})
test: (219, 11) Counter({1: 113, 0: 106})

train: (30319, 11) Counter({1: 15189, 0: 15130})
test: (12995, 11) Counter({0: 6527, 1: 6468})

train: (1180, 11) Counter({0: 597, 1: 583})
test: (506, 11) Counter({1: 260, 0: 246})

train: (31037, 11) Counter({1: 15519, 0: 15518})
test: (13303, 11) Counter({0: 6652, 1: 6651})

train: (29320, 11) Counter({0: 14669, 1: 14651})
test: (12566, 11) Counter({1: 6292, 0: 6274})



In [7]:
print(df.columns)
df.head()

Index(['Address', 'Brain Method', 'Extensive Coupling', 'Feature Envy',
       'Intensive Coupling', 'Shortcircuiting Method', 'Shotgun Surgery',
       'ALD', 'ATFD', 'CALIN', 'CC', 'CCL', 'CDISP', 'CEXT', 'CFG Cyclo',
       'CINT', 'CM', 'CYCLO', 'DUPSTRING', 'EDUPLINES', 'FANIN', 'FANOUT',
       'FANOUTCLASS', 'FDP', 'HDUPLINES', 'IDUPLINES', 'LAA', 'LOC',
       'MAXNESTING', 'NOAV', 'NOEU', 'NOLV', 'NOP', 'NOVRM',
       'NoOfExternalClients', 'NoOfInternalClients', 'PCR',
       'average call generality', 'Unnamed: 38', 'Name', 'Unnamed: 39'],
      dtype='object')


,Address,Brain Method,Extensive Coupling,Feature Envy,Intensive Coupling,Shortcircuiting Method,Shotgun Surgery,ALD,ATFD,CALIN,...,NOLV,NOP,NOVRM,NoOfExternalClients,NoOfInternalClients,PCR,average call generality,Unnamed: 38,Name,Unnamed: 39
0,example.Listener.main(java.lang.String[]),False,False,False,False,False,False,0,0,0,...,16,1,0,0,0,-1.0,1,NaN,NaN,NaN
1,"example.Listener.env(java.lang.String,java.lan...",False,False,False,False,False,False,0,0,0,...,1,2,0,0,0,-1.0,1,NaN,NaN,NaN
2,"example.Listener.arg(java.lang.String[],_GLOBA...",False,False,False,False,False,False,0,0,0,...,0,3,0,0,0,-1.0,1,NaN,NaN,NaN
3,example.Publisher.main(java.lang.String[]),False,False,False,False,False,False,0,0,0,...,17,1,0,0,0,-1.0,1,NaN,NaN,NaN
4,"example.Publisher.env(java.lang.String,java.la...",False,False,False,False,False,False,0,0,0,...,1,2,0,0,0,-1.0,1,NaN,NaN,NaN


In [12]:
print(Counter(df['Brain Method']), 
      Counter(df['Extensive Coupling']), 
      Counter(df['Feature Envy']), 
      Counter(df['Intensive Coupling']), 
      Counter(df['Shortcircuiting Method']))

Counter({False: 2471002, True: 15280}) Counter({False: 2483162, True: 3120}) Counter({False: 2463859, True: 22423}) Counter({False: 2478196, True: 8086}) Counter({False: 2480959, True: 5323})


In [12]:
x_st = 0
for x_end in list(range(500, 3369, 500))+[3369]:
    print(x_st, x_end)
    
    #x_st = x_end

0 500
0 1000
0 1500
0 2000
0 2500
0 3000
0 3369
